# Tensorflow基础API使用之基础数据类型

In [1]:
# 导入
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.1
pandas 0.25.3
sklearn 0.22.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


## 常量
### 1. 基础API--常量矩阵tf.constant

In [2]:
t = tf.constant([[1.,2.,3.],[4.,5.,6.]])

# index索引操作
print(t)
print(t[:,1:]) # 切片
print(t[...,1]) # 取出第2列

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [3]:
# op算子操作
print(t+10) # 加法
print(tf.square(t)) # 求平方
print(t @ tf.transpose(t)) # t乘t的转置

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [4]:
# numpy conversion : tf对象和numpy转化
print(t.numpy())
print(np.square(t))
np_t = np.array([[1.,2.,3.],[4.,5.,6.]])
print(tf.constant(np_t))

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [5]:
# Scalars ：0维数字
t = tf.constant(2.718)
print(t.numpy())
print(t.shape)

2.718
()


### 2. 基础API--字符串tf.strings

In [7]:
# strings
t = tf.constant('cafe')
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t,unit='UTF8_CHAR'))
print(tf.strings.unicode_decode(t,'UTF8'))

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [8]:
# string array
t = tf.constant(['cafe','coffee','咖啡'])
print(tf.strings.length(t,unit='UTF8_CHAR'))
r = tf.strings.unicode_decode(t,'UTF8')
print(r)

tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


### 3. 基础API--不等长矩阵tf.ragged.constant

In [9]:
r = tf.ragged.constant([[11,12],[21,22,23],[],[41]])

# index op 索引操作
print(r)
print(r[1])
print(r[1:2]) # 切片（左闭右开）

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23]]>


In [10]:
# ops on ragged tensor
r2 = tf.ragged.constant([[51,52],[],[71]])
print(tf.concat([r,r2], axis=0)) # 横向拼接可，竖向拼接因为维度不一样而不可

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [71]]>


In [12]:
r3 = tf.ragged.constant([[13,14],[15],[],[42,43]])
print(tf.concat([r,r3], axis=1)) # 竖向拼接

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 15], [], [41, 42, 43]]>


In [13]:
print(r.to_tensor()) # 将ragged tensor转化为tensor

tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


### 4. 基础API--稀疏矩阵tf.SparseTensor

In [14]:
# sparse tensor
s = tf.SparseTensor(indices=[[0,1],[1,0],[2,3]],
                    values=[1.,2.,3.],
                    dense_shape=[3,4])
print(s)
print(tf.sparse.to_dense(s)) # 将稀疏矩阵转化为密集矩阵

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [16]:
# ops on sparse tensors
s2 = s * 2.0 # 乘法
print(s2)

try:
    s3 = s + 1
except TypeError as ex:
    print(ex)
    
s4 = tf.constant([[10.,20.],[30.,40.],[50.,60.],[70.,80.]])
print(tf.sparse.sparse_dense_matmul(s,s4)) # 将稀疏矩阵和密集矩阵相乘

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [17]:
s5 = tf.SparseTensor(indices=[[0,2],[0,1],[2,3]],
                    values=[1.,2.,3.],
                    dense_shape=[3,4])
print(s5)
s6 = tf.sparse.reorder(s5)
print(tf.sparse.to_dense(s6)) # to_dense方法只能在排好序的时候调用

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


## 变量
### 5. 基础API--变量tf.Variable

In [18]:
# Variables
v = tf.Variable([[1.,2.,3.], [4.,5.,6.]])
print(v)
print(v.value()) # 将变量变为 tensor张量
print(v.numpy()) # 转化为值

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [19]:
# assign value 重新赋值
v.assign(2*v) # 整体重新赋值
print(v.numpy())
v[0,1].assign(42) # 具体某个值赋值
print(v.numpy())
v[1].assign([7.,8.,9.]) # 某一行赋值
print(v.numpy())

[[ 2.  4.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 7.  8.  9.]]


In [20]:
# 不能使用等号赋值
try:
    v[1] = [7.,8.,9.] 
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
